In [13]:
import numpy as np
import torch
import torch.nn.functional as F
from sklearn import utils
from sklearn.linear_model import LinearRegression


In [14]:
#load the filtered data
ftr = np.load('features_f.npy', allow_pickle=True)
lbl = np.load('labels_f.npy', allow_pickle=True)

In [15]:
#each array in (features) is a row in data
ftr[0]

array(['Chad Hurley', 'YouTube', 0.0, 0.0,
       'Indiana University of Pennsylvania', 'Design', nan, nan, nan, nan,
       0.0, 4.0, 1.0, 1.0, 0.0, 0.0, 0, 0,
       'https://www.crunchbase.com/person/chad-hurley',
       'http://www.linkedin.com/profile/view?id=5711'], dtype=object)

In [16]:
#transpose it so that each column is now an array
ftr = ftr.transpose()[1:18]
lbls = lbl.transpose()

In [17]:
ftr[0] #each column is now an array

array(['YouTube', 'Goop Inc.', 'Inside.com', 'Nest Labs', 'Automattic',
       'Tempus', 'Instagram', 'LendingHome', 'Asana', 'WhatsApp', 'Talla',
       'Kiip', 'Sift Science', 'Droom Technology', 'ClassPass', 'Oculus',
       'Krux', 'Brandless', 'Twitch', 'Engagio', 'Fabletics', 'Moat',
       'Tenable Network Security, Inc.', 'Olapic', 'Patreon', 'WayUp',
       'Robinhood', 'WITHIN (VR/AR)', 'Ripcord', 'Planet', 'HealthTap',
       'DoorDash', 'Tamr', 'Mashable', 'The Climate Corporation', 'ipsy',
       'Casper Sleep', 'Doppler Labs', 'Accompany', 'Anonyome Labs',
       'MasterClass', 'Qualtrics', 'OfferUp', 'Glossier', 'Nextdoor',
       '21 Inc', 'Mattermark', 'Honor', 'Away', 'Luxe', 'Dropcam',
       'Nauto, Inc.', 'EdCast', 'Gusto', 'Quip', 'StyleSeat', 'Yik Yak',
       'Robinhood', 'x.ai', 'AltSchool', 'Vungle', 'AirMap', 'Tile',
       'TINYhr', 'Andela', 'Drift', 'Quote Roller', 'Drawbridge Inc.',
       'Squarespace', 'Cotopaxi', 'Mixpanel', 'Lumity, Inc.', 'Freshly',


In [18]:
#we need a label arary with success or failure labels
#we are using valuation increased column to define that. if valution increased is greater than zero, than startup was a success
#if not than its a failure
# valuation increased has  undefined values like '#VALUE!' and '#DIV/0!', so we need to filter them

lbl[5] #valuation increased coulmn

lbl_v = lbls[5] #save valuation increase in lbl_v

lbl_v

lbls[4] #b valuation column

lbl_av = lbls[3] #a valuation
lbl_bv = lbls[4] #b valuation


# a loop that make sure that all the undefined values in valuation increased variable such as '#Value!' and '#DIV/0!' are rectified
# and repalced with the right values i.e 'b valuation / a valuation'
# the final lbl_v will have values as 0 or 1 as failed or succesful startup respectively.
for idx, data in enumerate(lbl_v):
    if data == '#VALUE!': #if undefined value
        try: #check if b_val/a_val is possible
            temp = float(lbl_bv[idx].replace(',',''))/ float(lbl_av[idx].replace(',',''))
            if temp>float(0): # b_val/#a_val > 0
                lbl_v[idx] = float(1) #startup is a success
            else:
                lbl_v[idx]= float(0) #else failure
        except: #if the code produces error as a_val is zero and temp variable is not possible because dividing by zero is nan
            lbl_v[idx] = float(0) #startup is a failure :)
    #same goes for #DIV/0! values in labels
    elif data == '#DIV/0!':
        try:
            temp = float(lbl_bv[idx].replace(',',''))/ float(lbl_av[idx].replace(',',''))
            if temp>float(0):
                lbl_v[idx] = float(1)
            else:
                lbl_v[idx] = float(0)
        except:
            lbl_v[idx] = float(0)
            
    #but if the 'valuation increased' value exists then:
    elif float(data.replace(',','')) > float(0): #check if that number is greater than zero
        lbl_v[idx] = float(1) #if yes then its a succes 
    else:
        lbl_v[idx] = float(0) #else failure
        

lbl_v = lbl_v.astype(float) #save the filtered/refined lbl_v as float datatype values as zero or one

In [19]:
np_ftrs = ftr

In [20]:
for i in range(len(np_ftrs)):
    try:
        np_ftrs[i] = np_ftrs[i].astype(int)
    except:
        for idx, ele in enumerate(np_ftrs[i]):
            if ele!= 0.0 and ele!= 1.0:
                np_ftrs[i][idx] = 0.0
        np_ftrs[i] = np_ftrs[i].astype(int)
    

In [21]:
np_ftrs[0]=np.unique(np_ftrs[0], return_inverse=True)[1] #convert 'string' features into 'integers' beacuse model does not take string values
print(np_ftrs[0]) #printed 'previous company' coulmn in numbers PS we removed this column entirely in the end due to its unstability

# it was seen that from column 3 to 8, they have nan values which are undefined values, we need to convert them to strings
for i in range(6):
    for idx,data in enumerate(np_ftrs[i+3]):
        if type(data)!= str:
            np_ftrs[i+3][idx]= 'nan'
# now change those column string values into numbers too
for i in range(6):
    np_ftrs[i+3] = np.unique(np_ftrs[i+3], return_inverse=True)[1]
    print(np_ftrs[i+3])

#now for all the columns in features, 
for i in range(len(np_ftrs)):
    try: #convert those columns into integer datatype
        np_ftrs[i] = np_ftrs[i].astype(int)
    except: #if cannot, that means they are the columns with 0 and 1 values and they have a string type 'nan' value, so convert 'nan' into float '0.0' and then convert the whole column into integer
        for idx, ele in enumerate(np_ftrs[i]):
            if ele!= 0.0 and ele!= 1.0:
                np_ftrs[i][idx] = 0.0
        np_ftrs[i] = np_ftrs[i].astype(int)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [22]:
for count in range(len(np_ftrs)):
    print(count, np.corrcoef(list(np_ftrs[count]), lbl_v)) #find the coreration for every column with lbl_v

0 [[nan nan]
 [nan  1.]]
1 [[ 1.         -0.05982415]
 [-0.05982415  1.        ]]
2 [[1.         0.10611566]
 [0.10611566 1.        ]]
3 [[nan nan]
 [nan  1.]]
4 [[nan nan]
 [nan  1.]]
5 [[nan nan]
 [nan  1.]]
6 [[nan nan]
 [nan  1.]]
7 [[nan nan]
 [nan  1.]]
8 [[nan nan]
 [nan  1.]]
9 [[1.         0.08313118]
 [0.08313118 1.        ]]
10 [[ 1.         -0.03555541]
 [-0.03555541  1.        ]]
11 [[1.         0.19909812]
 [0.19909812 1.        ]]
12 [[1.         0.10722047]
 [0.10722047 1.        ]]
13 [[1.         0.00179018]
 [0.00179018 1.        ]]
14 [[ 1.         -0.08694233]
 [-0.08694233  1.        ]]
15 [[1.         0.11022688]
 [0.11022688 1.        ]]
16 [[1.         0.06666861]
 [0.06666861 1.        ]]


/Users/idantabachnik/Documents/startup/.venv/lib/python3.10/site-packages/numpy/lib/function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/Users/idantabachnik/Documents/startup/.venv/lib/python3.10/site-packages/numpy/lib/function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
